<a href="https://colab.research.google.com/github/FMekujee27/RNNs/blob/main/3_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import scikeras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import reservoirpy as rpy
import torch
import datetime
import seaborn as sns
import tensorflow as tf
import random
import yfinance as yf
import time
import keras
import ta
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras import layers, models
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Dense, SimpleRNN, Dropout, GRU
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error
from reservoirpy.nodes import Reservoir, Ridge
from sklearn.preprocessing import StandardScaler
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from scipy import stats
from sklearn.base import BaseEstimator, RegressorMixin
from scipy.stats import uniform

In [11]:
#Load the data
df=pd.read_excel("/content/Scores.xlsx")
#Set Date column as an index
df.set_index("Date",inplace=True)
df.index = pd.to_datetime(df.index)
#Removing headline
df=df.drop(columns=["Headline"])
df.index = pd.to_datetime(df.index)
#Daily average sentiment score
df = df.resample("D").mean()
df=df.dropna()
df.index=df.index.strftime("%d-%m-%Y")
df.head()

,sentiment_score
Date,
15-06-2011,-0.305849
16-06-2011,-0.162089
18-06-2011,-0.013107
19-06-2011,-0.019240
20-06-2011,-0.638288


In [12]:
#WTI price data from yahoo finance
df1 = yf.download('CL=F', start='2011-06-15', end='2024-06-21')
df1=df1[["Open","Adj Close"]]
df1.index = pd.to_datetime(df1.index)
df1.index=df1.index.strftime("%d-%m-%Y")
#Joining the bews and price data frame on index
df=df1.join(df)
df=df.dropna()
df.head()
df=df.dropna()
df.head()
M=df["Adj Close"].mean()
S=df["Adj Close"].std()
#Convert the values to standard score
df["Open*Score"]=df["Open"]*(df["sentiment_score"].shift(1))
df["lag price"]=df["Adj Close"].shift(1)
df=df.drop(columns=["Open","sentiment_score"])
df=df.dropna()
df.head()

[*********************100%%**********************]  1 of 1 completed


,Adj Close,Open*Score,lag price
Date,,,
16-06-2011,94.949997,-29.144371,94.809998
20-06-2011,93.260002,-15.041867,94.949997
21-06-2011,93.400002,-59.673567,93.260002
22-06-2011,95.410004,-19.420126,93.400002
23-06-2011,91.019997,-5.088924,95.410004


In [14]:
#Summary statistics
df.describe().round(4)

,Adj Close,Open*Score,lag price
count,3148.0000,3148.0000,3148.0000
mean,69.8172,-6.5566,69.8212
std,21.9082,21.7981,21.9116
min,-37.6300,-105.5298,-37.6300
25%,51.4275,-18.2998,51.4275
50%,68.7050,-5.2338,68.7050
75%,88.8425,3.7576,88.8625
max,123.7000,104.3721,123.7000


In [88]:
import plotly.express as px

for x in df.columns:
    fig = px.line(df, x=df.index, y=x, title=f"Time Series plot of {x}", labels={"x": "Date", "y": "Value"})
    fig.update_xaxes(tickangle=0,nticks=8)
    fig.update_traces(line_color="skyblue")
    fig.show()


In [17]:
# Create a data frame
results = pd.DataFrame(columns=["Statistic", "P-Value", "Normality"])

# Perform the Shapiro-Wilk test
for x in df.columns:
    s, p = stats.shapiro(df[x])
    normality = p > 0.05
    results.loc[x] = [s, p, normality]
results



,Statistic,P-Value,Normality
Adj Close,0.971975,2.276803e-24,False
Open*Score,0.965484,8.052365e-27,False
lag price,0.971914,2.149441e-24,False


In [18]:
#Fitting standard scaler
scaler=StandardScaler()
for x in df.columns:
    df[x]=scaler.fit_transform(df[[x]])
df.head()

,Adj Close,Open*Score,lag price
Date,,,
16-06-2011,1.147369,-1.036394,1.140617
20-06-2011,1.070217,-0.389330,1.147007
21-06-2011,1.076609,-2.437162,1.069867
22-06-2011,1.168370,-0.590217,1.076257
23-06-2011,0.967956,0.067340,1.168004


In [76]:
# Create the heatmap
fig = px.imshow(df.corr(), color_continuous_scale="Blues", title="Heatmap")

# Show the heatmap
fig.show()

In [29]:
#Feature matrix
x=df.drop("Adj Close",axis=1)
#Traget vector
y=df["Adj Close"]
#Converting to numpy array
x,y=np.array(x),np.array(y)
#Training and test dataset
x_train,x_test=x[:-48],x[-48:]
y_train,y_test=y[:-48],y[-48:]
#Reshaping data
x_train=x_train.reshape(x_train.shape[0],1,x_train.shape[1])
y_train=y_train.reshape(y_train.shape[0],1)
y_test=y_test.reshape(y_test.shape[0],1)
x_test=x_test.reshape(x_test.shape[0],1,x_test.shape[1])

In [33]:
#Parametr grid for RNN,LSTM and GRU
param_grid = {
    "model__rnn_units_0": [50,100,150,200,250,32,64,128,256,512],
    "model__rnn_units_1": [50,100,150,200,250,32,64,128,256,512],
    "model__rnn_units_2": [50,100,150,200,250,32,64,128,256,512],
    "model__rnn_units_3": [50,100,150,200,250,32,64,128,256,512],
    "model__rnn_units_4": [50,100,150,200,250,32,64,128,256,512],
    "model__rnn_activation_0": ["relu", "tanh", "linear", "sigmoid"],
    "model__rnn_activation_1": ["relu", "tanh", "linear", "sigmoid"],
    "model__rnn_activation_2": ["relu", "tanh", "linear", "sigmoid"],
    "model__rnn_activation_3": ["relu", "tanh", "linear", "sigmoid"],
    "model__rnn_activation_4": ["relu", "tanh", "linear", "sigmoid"],
    "model__dropout": [True, False],
    "model__dense_activation": ["relu", "tanh", "linear", "sigmoid"],
    "model__learning_rate": uniform(0.1, 0.8)
}

In [31]:
#RNN model
def build_model(rnn_units_0=32, rnn_units_1=32, rnn_units_2=32, rnn_units_3=32, rnn_units_4=32,
                rnn_activation_0="relu", rnn_activation_1="relu", rnn_activation_2="relu",
                rnn_activation_3="relu", rnn_activation_4="relu", dropout=False, dense_activation="relu",learning_rate=1e-3):
    model = models.Sequential()

    # Add an Input layer with the correct shape (timesteps, features)
    model.add(layers.Input(shape=(x_train.shape[1], x_train.shape[2])))

    # Add 5 SimpleRNN layers
    model.add(layers.SimpleRNN(units=rnn_units_0, activation=rnn_activation_0, return_sequences=True))
    model.add(layers.SimpleRNN(units=rnn_units_1, activation=rnn_activation_1, return_sequences=True))
    model.add(layers.SimpleRNN(units=rnn_units_2, activation=rnn_activation_2, return_sequences=True))
    model.add(layers.SimpleRNN(units=rnn_units_3, activation=rnn_activation_3, return_sequences=True))
    model.add(layers.SimpleRNN(units=rnn_units_4, activation=rnn_activation_4, return_sequences=False))

    if dropout:
        model.add(layers.Dropout(rate=0.1))

    model.add(layers.Dense(1, activation=dense_activation))
    model.compile(optimizer="adam", loss="mean_absolute_error", metrics=["accuracy"])
    return model


# Wrap the model using KerasRegressor
model = KerasRegressor(model=build_model, epochs=10, batch_size=32, verbose=0)

# Perform the search using RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, cv=3, verbose=2, n_jobs=-1,random_state=42)
random_search_result = random_search.fit(x_train, y_train)

# Print the best hyperparameters
best_hps = random_search_result.best_params_
print("Best hyperparameters:", best_hps)

# Extract the best hyperparameters
best_model_params = {k.replace("model__", ""): v for k, v in best_hps.items()}

# Build the model with the best hyperparameters
RNN_model = build_model(**best_model_params)

# Fit the model with the best hyperparameters
T1 = time.time()
RNN_model.fit(x_train, y_train,validation_split=0.2)
T2 = time.time()


# Run time
T_RNN = T2 - T1
Y_RNN = RNN_model.predict(x_test)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best hyperparameters: {'model__dense_activation': 'linear', 'model__dropout': False, 'model__learning_rate': 0.1926952476201038, 'model__rnn_activation_0': 'linear', 'model__rnn_activation_1': 'relu', 'model__rnn_activation_2': 'sigmoid', 'model__rnn_activation_3': 'sigmoid', 'model__rnn_activation_4': 'tanh', 'model__rnn_units_0': 50, 'model__rnn_units_1': 64, 'model__rnn_units_2': 64, 'model__rnn_units_3': 128, 'model__rnn_units_4': 250}
78/78 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - accuracy: 0.0000e+00 - loss: 0.8049 - val_accuracy: 0.0000e+00 - val_loss: 0.2458
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 553ms/step


In [32]:
#LSTM model
def build_model(rnn_units_0=32, rnn_units_1=32, rnn_units_2=32, rnn_units_3=32, rnn_units_4=32,
                rnn_activation_0="relu", rnn_activation_1="relu", rnn_activation_2="relu",
                rnn_activation_3="relu", rnn_activation_4="relu", dropout=False, dense_activation="relu",learning_rate=1e-3):
    model = models.Sequential()

    # Add an Input layer with the correct shape (timesteps, features)
    model.add(layers.Input(shape=(x_train.shape[1], x_train.shape[2])))

    # Add 5 LSTM layers
    model.add(layers.LSTM(units=rnn_units_0, activation=rnn_activation_0, return_sequences=True))
    model.add(layers.LSTM(units=rnn_units_1, activation=rnn_activation_1, return_sequences=True))
    model.add(layers.LSTM(units=rnn_units_2, activation=rnn_activation_2, return_sequences=True))
    model.add(layers.LSTM(units=rnn_units_3, activation=rnn_activation_3, return_sequences=True))
    model.add(layers.LSTM(units=rnn_units_4, activation=rnn_activation_4, return_sequences=False))

    if dropout:
        model.add(layers.Dropout(rate=0.1))

    model.add(layers.Dense(1, activation=dense_activation))
    model.compile(optimizer="adam", loss="mean_absolute_error", metrics=["accuracy"])
    return model


# Wrap the model using KerasRegressor
model = KerasRegressor(model=build_model, epochs=10, batch_size=32, verbose=0)

# Perform the search using RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, cv=3, verbose=2, n_jobs=-1,random_state=42)
random_search_result = random_search.fit(x_train, y_train)

# Print the best hyperparameters
best_hps = random_search_result.best_params_
print("Best hyperparameters:", best_hps)

# Extract the best hyperparameters
best_model_params = {k.replace("model__", ""): v for k, v in best_hps.items()}

# Build the model with the best hyperparameters
LSTM_model = build_model(**best_model_params)

# Fit the model with the best hyperparameters
T1 = time.time()
LSTM_model.fit(x_train, y_train,validation_split=0.2)
T2 = time.time()


# Run time
T_LSTM = T2 - T1
Y_LSTM = LSTM_model.predict(x_test)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best hyperparameters: {'model__dense_activation': 'linear', 'model__dropout': False, 'model__learning_rate': 0.8605714451279329, 'model__rnn_activation_0': 'linear', 'model__rnn_activation_1': 'sigmoid', 'model__rnn_activation_2': 'relu', 'model__rnn_activation_3': 'relu', 'model__rnn_activation_4': 'linear', 'model__rnn_units_0': 512, 'model__rnn_units_1': 150, 'model__rnn_units_2': 64, 'model__rnn_units_3': 128, 'model__rnn_units_4': 250}
78/78 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - accuracy: 0.0000e+00 - loss: 0.8452 - val_accuracy: 0.0000e+00 - val_loss: 0.6702
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step


In [34]:
#GRU model
def build_model(rnn_units_0=32, rnn_units_1=32, rnn_units_2=32, rnn_units_3=32, rnn_units_4=32,
                rnn_activation_0="relu", rnn_activation_1="relu", rnn_activation_2="relu",
                rnn_activation_3="relu", rnn_activation_4="relu", dropout=False, dense_activation="relu",learning_rate=1e-3):
    model = models.Sequential()

    # Add an Input layer with the correct shape (timesteps, features)
    model.add(layers.Input(shape=(x_train.shape[1], x_train.shape[2])))

    # Add 5 GRU layers
    model.add(layers.GRU(units=rnn_units_0, activation=rnn_activation_0, return_sequences=True))
    model.add(layers.GRU(units=rnn_units_1, activation=rnn_activation_1, return_sequences=True))
    model.add(layers.GRU(units=rnn_units_2, activation=rnn_activation_2, return_sequences=True))
    model.add(layers.GRU(units=rnn_units_3, activation=rnn_activation_3, return_sequences=True))
    model.add(layers.GRU(units=rnn_units_4, activation=rnn_activation_4, return_sequences=False))

    if dropout:
        model.add(layers.Dropout(rate=0.1))

    model.add(layers.Dense(1, activation=dense_activation))
    model.compile(optimizer="adam", loss="mean_absolute_error", metrics=["accuracy"])
    return model


# Wrap the model using KerasRegressor
model = KerasRegressor(model=build_model, epochs=10, batch_size=32, verbose=0)

# Perform the search using RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, cv=3, verbose=2, n_jobs=-1,random_state=42)
random_search_result = random_search.fit(x_train, y_train)

# Print the best hyperparameters
best_hps = random_search_result.best_params_
print("Best hyperparameters:", best_hps)

# Extract the best hyperparameters
best_model_params = {k.replace("model__", ""): v for k, v in best_hps.items()}

# Build the model with the best hyperparameters
GRU_model = build_model(**best_model_params)

# Fit the model with the best hyperparameters
T1 = time.time()
GRU_model.fit(x_train, y_train,validation_split=0.2)
T2 = time.time()


# Run time
T_GRU = T2 - T1
Y_GRU = GRU_model.predict(x_test)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best hyperparameters: {'model__dense_activation': 'linear', 'model__dropout': False, 'model__learning_rate': 0.1926952476201038, 'model__rnn_activation_0': 'linear', 'model__rnn_activation_1': 'relu', 'model__rnn_activation_2': 'sigmoid', 'model__rnn_activation_3': 'sigmoid', 'model__rnn_activation_4': 'tanh', 'model__rnn_units_0': 50, 'model__rnn_units_1': 64, 'model__rnn_units_2': 64, 'model__rnn_units_3': 128, 'model__rnn_units_4': 250}
78/78 ━━━━━━━━━━━━━━━━━━━━ 13s 29ms/step - accuracy: 0.0000e+00 - loss: 0.8783 - val_accuracy: 0.0000e+00 - val_loss: 0.2243


1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 796ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 814ms/step


In [35]:
#QRNN model
def build_model(filters=32, kernel_size=3, dense_units=32, learning_rate=1e-3):
    inputs = Input(shape=(x_train.shape[1], x_train.shape[2]))
    if kernel_size > x_train.shape[1]:
        kernel_size = x_train.shape[1]
    x = layers.Conv1D(filters=filters, kernel_size=kernel_size, activation="relu")(inputs)
    x = layers.Lambda(lambda x: tf.reduce_mean(x, axis=1))(x)
    x = layers.Dense(dense_units, activation="relu")(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="mean_absolute_error", metrics=["accuracy"])

    return model



# Wrap the model using KerasRegressor
model = KerasRegressor(model=build_model, epochs=10, batch_size=32, verbose=0)

# Define the parameter grid
param_grid = {
    "model__filters": [32, 64, 128],
    "model__kernel_size": [2, 3, 4, 5],  # Ensure these values are valid for your input shape
    "model__dense_units": [50,100,150,200,250,32,64,128,256,512],
    "model__learning_rate": uniform(0.1, 0.8)
}

# Perform the search using RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, cv=3, verbose=2, n_jobs=-1,random_state=42)
random_search_result = random_search.fit(x_train, y_train)


# Print the best hyperparameters
best_hps = random_search_result.best_params_
print("Best hyperparameters:", best_hps)

# Extract the best hyperparameters
best_filters = best_hps["model__filters"]
best_kernel_size = best_hps["model__kernel_size"]
best_dense_units = best_hps["model__dense_units"]
best_learning_rate = best_hps["model__learning_rate"]

# Build the model with the best hyperparameters
QRNN_model = build_model(filters=best_filters, kernel_size=best_kernel_size, dense_units=best_dense_units, learning_rate=best_learning_rate)

# Fit the model with the best hyperparameters
T1 = time.time()
QRNN_model.fit(x_train, y_train, epochs=10, validation_split=0.2)
T2=time.time()
# Run time
T_QRNN = T2 - T1
Y_QRNN = QRNN_model.predict(x_test)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best hyperparameters: {'model__dense_units': 50, 'model__filters': 64, 'model__kernel_size': 3, 'model__learning_rate': 0.10565304417577392}
Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.0000e+00 - loss: 0.4302 - val_accuracy: 0.0000e+00 - val_loss: 0.0895
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0518 - val_accuracy: 0.0000e+00 - val_loss: 0.0819
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0502 - val_accuracy: 0.0000e+00 - val_loss: 0.0795
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0481 - val_accuracy: 0.0000e+00 - val_loss: 0.0784
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0485 - val_accuracy: 0.0000e+00 - val_loss: 0.0786
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0481 - val_accuracy: 0.0000e+00 - val_loss: 0

In [36]:
#Reshaping the data
x_train=x_train.reshape(x_train.shape[0],-1)
x_test=x_test.reshape(x_test.shape[0],-1)

In [37]:
#ESN model
class ESNRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, units=100, lr=0.5, sr=0.9, ridge=1e-7):
        self.units = units
        self.lr = lr
        self.sr = sr
        self.ridge = ridge
        self.esn_model = None

    def fit(self, X, y):
        reservoir = Reservoir(units=self.units, lr=self.lr, sr=self.sr)
        readout = Ridge(ridge=self.ridge)
        self.esn_model = reservoir >> readout
        self.esn_model.fit(X, y)
        return self

    def predict(self, X):
        return self.esn_model.run(X)

# Define the model
esn_model = ESNRegressor()

# Define the hyperparameter space
param_grid = {
    "units": [50,100,150,200,250,32,64,128,256,512],
    "lr": uniform(0.1, 0.8),
    "sr": uniform(-1, 1),
    "ridge": uniform(1e-7, 1e-5)
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=esn_model, param_distributions=param_grid, n_iter=10, cv=3, verbose=2, n_jobs=-1,scoring="neg_mean_absolute_error",random_state=42)


# Perform the search
random_search.fit(x_train, y_train)

# Get the best hyperparameters
best_params = random_search.best_params_


# Evaluate the model with the best hyperparameters
ESN_model = random_search.best_estimator_

# Fit the model with the best hyperparameters
T1 = time.time()
ESN_model.fit(x_train, y_train)
T2 = time.time()
#Run time
T_ESN = T2 - T1
Y_ESN=ESN_model.predict(x_test)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


Running Model-0:   0%|          | 0/1 [00:00<?, ?it/s]
Running Model-0: 355it [00:00, 3546.57it/s]           
Running Model-0: 733it [00:00, 3671.49it/s]
Running Model-0: 1101it [00:00, 3168.29it/s]
Running Model-0: 1458it [00:00, 3311.75it/s]
Running Model-0: 1854it [00:00, 3530.37it/s]
Running Model-0: 2226it [00:00, 3590.53it/s]
Running Model-0: 2589it [00:00, 3545.12it/s]
Running Model-0: 3100it [00:00, 3195.66it/s]
Running Model-0: 100%|██████████| 1/1 [00:01<00:00,  1.02s/it]


Fitting node Ridge-0...


Running Model-1:   0%|          | 0/1 [00:00<?, ?it/s]
Running Model-1: 233it [00:00, 2327.98it/s]           
Running Model-1: 466it [00:00, 2295.00it/s]
Running Model-1: 712it [00:00, 2365.82it/s]
Running Model-1: 949it [00:00, 2209.49it/s]
Running Model-1: 1226it [00:00, 2398.96it/s]
Running Model-1: 1468it [00:00, 2378.41it/s]
Running Model-1: 1708it [00:00, 2305.33it/s]
Running Model-1: 1940it [00:00, 2292.65it/s]
Running Model-1: 2170it [00:00, 2164.14it/s]
Running Model-1: 2388it [00:01, 2004.27it/s]
Running Model-1: 2592it [00:01, 1849.90it/s]
Running Model-1: 2784it [00:01, 1866.95it/s]
Running Model-1: 3100it [00:01, 2028.26it/s]
Running Model-1: 100%|██████████| 1/1 [00:01<00:00,  1.58s/it]


Fitting node Ridge-1...


Running Model-1: 48it [00:00, 2582.67it/s]            


In [38]:
#Creating prediction data fraem
df1=df.iloc[-48:,0].to_frame()
df1 = df1.rename(columns={"Adj Close": "Actual"})
df1["RNN"]=Y_RNN
df1["LSTM"]=Y_LSTM
df1["GRU"]=Y_GRU
df1["QRNN"]=Y_QRNN
df1["ESN"]=Y_ESN
#Unscalling the values
df1=((S*df1)+M)
df1.head()

,Actual,RNN,LSTM,GRU,QRNN,ESN
Date,,,,,,
12-04-2024,85.671215,91.062408,70.522011,88.664276,84.966988,84.481625
15-04-2024,85.421163,92.354385,70.740341,88.533195,85.558876,85.433266
16-04-2024,85.371150,92.412315,70.779884,88.193703,85.283241,85.547405
17-04-2024,82.700600,91.194992,70.565552,89.118896,85.101730,84.849204
18-04-2024,82.740609,89.016357,70.118446,86.702133,82.770508,82.107847


In [95]:
#Plot the data frame of predictions
fig = px.line(df1, x=df1.index, y=df1.columns, title='Actual vs Predicted Prices', labels={"x": "Date", "value": "Price"})
fig.update_layout(legend_title_text="Model")
fig.update_xaxes(tickangle=0,nticks=8)
# Show the plot
fig.show()

In [55]:
#Perfomance matrices
def met(df):

    actual = df.iloc[:, 0]
    values = {

        "Model": [],
        "MSE": [],
        "MAPE": [],
        'R-squared': []
    }
    for c in df.columns[1:]:
        y_a = actual
        y_p = df[c]
        mse = mean_squared_error(y_a, y_p)
        mape = mean_absolute_percentage_error(y_a,y_p)
        r_squared = r2_score(y_a, y_p)
        values["Model"].append(c)
        values["MSE"].append(mse)
        values["MAPE"].append(mape)
        values["R-squared"].append(r_squared)

    values = pd.DataFrame(values)
    values.set_index("Model",inplace=True)
    return values
df2=met(df1)
df2.round(4)

,MSE,MAPE,R-squared
Model,,,
RNN,38.4407,0.0759,-3.5023
LSTM,110.5908,0.1280,-11.9527
GRU,26.0247,0.0624,-2.0481
QRNN,1.2990,0.0109,0.8479
ESN,1.2212,0.0106,0.8570


In [100]:
#Creating dataframe of model fitting time
df3=pd.DataFrame()
df3["Time"]=[T_RNN,T_LSTM,T_GRU,T_QRNN,T_ESN]
df3.index=["RNN","LSTM","GRU","QRNN","ESN"]
#Plotting the model run time
fig = px.bar(df3, x=df3.index, y=df3.columns[0], color_discrete_sequence=["skyblue"], title="Model Fitting Time",labels={"index": "Model", "Time": "Time in seconds"})
fig.show()

In [131]:
# Define the RSI Strategy
class RSI(Strategy):
    n1 = 14  # RSI period
    upper_bound = 70
    lower_bound = 30

    def init(self):
        close_series = pd.Series(self.data.Close)
        self.rsi = self.I(lambda: ta.momentum.RSIIndicator(close_series, self.n1).rsi())

    def next(self):
        if crossover(self.rsi, self.lower_bound):
            self.buy()
        elif crossover(self.rsi, self.upper_bound):
            self.sell()

# Define the MACD Strategy
class MACD(Strategy):
    fast_period = 12
    slow_period = 26
    signal_period = 9

    def init(self):
        close_series = pd.Series(self.data.Close)
        self.macd = self.I(lambda: ta.trend.MACD(close_series, self.fast_period, self.slow_period, self.signal_period).macd_diff())

    def next(self):
        if crossover(self.macd, 0):
            self.buy()
        elif crossover(0, self.macd):
            self.sell()

# Define the SMA Crossover Strategy
class SMA(Strategy):
    n1 = 5
    n2 = 20

    def init(self):
        close_series = pd.Series(self.data.Close)
        self.sma1 = self.I(lambda: ta.trend.SMAIndicator(close_series, self.n1).sma_indicator())
        self.sma2 = self.I(lambda: ta.trend.SMAIndicator(close_series, self.n2).sma_indicator())

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()

# Define the EMA Crossover Strategy
class EMA(Strategy):
    n1 = 5
    n2 = 20

    def init(self):
        close_series = pd.Series(self.data.Close)
        self.ema1 = self.I(lambda: ta.trend.EMAIndicator(close_series, self.n1).ema_indicator())
        self.ema2 = self.I(lambda: ta.trend.EMAIndicator(close_series, self.n2).ema_indicator())

    def next(self):
        if crossover(self.ema1, self.ema2):
            self.buy()
        elif crossover(self.ema2, self.ema1):
            self.sell()
# Define the Buy and Hold Strategy
class BuyAndHold(Strategy):
    def init(self):
        self.buy()

    def next(self):
        pass

In [132]:
# Function to perform backtest for a given strategy and column
def dat(strategy, x):
    df = yf.download("CL=F", start="2024-04-12", end="2024-06-21")
    df.drop(columns=["Adj Close", "Close"], inplace=True)
    back = pd.DataFrame(df1[x])
    back.index = df.index
    df["Close"] = back[x]
    df = df[["Open", "High", "Low", "Close", "Volume"]]
    bt = Backtest(df, strategy, cash=10_000, commission=0.02)
    stats = bt.run()
    return_percent = stats["Return [%]"]
    sharpe_ratio = stats["Sharpe Ratio"]
    data = {
        "Return [%]": [return_percent],
        "Sharpe Ratio": [sharpe_ratio],
    }
    stats_df = pd.DataFrame(data)
    return stats_df

# Function to perform backtests for all strategies and columns
def final(y):
    result = pd.DataFrame()
    strategies = [RSI, MACD, SMA, EMA, BuyAndHold]
    model_names = ["Actual","RNN", "LSTM", "GRU", "QRNN", "ESN"]
    for x in y.columns:
        for strategy in strategies:
            x_values = dat(strategy, x)
            x_values.index = [f"{strategy.__name__} on {x}"]
            result = pd.concat([result, x_values])
    result.index = pd.MultiIndex.from_product([model_names, [s.__name__ for s in strategies]], names=["Values", "Strategy"])
    return result

Result = final(df1)
Result

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Return [%]  Sharpe Ratio
Values Strategy                            
Actual RSI           0.867634      0.224610
       MACD          7.945872      2.574418
       SMA           2.500997      1.671213
       EMA           2.500997      1.671213
       BuyAndHold   -4.563202      0.000000
RNN    RSI           2.245991      0.357598
       MACD          9.597073      1.568283
       SMA           1.823288      0.437938
       EMA           1.823288      0.437938
       BuyAndHold  -11.722347      0.000000
LSTM   RSI           2.245991      0.203339
       MACD          9.597073      0.808680
       SMA           1.823288      0.168223
       EMA           1.823288      0.168223
       BuyAndHold   15.223128      1.264826
GRU    RSI           2.016500      0.368812
       MACD          9.597073      2.005552
       SMA           2.500997      0.834177
       EMA           1.823288      0.681155
       BuyAndHold   -7.915226      0.000000
QRNN   RSI           1.917247      0.465927
       MACD          9.597073      3.282742
       SMA           1.823288      1.115818
       EMA           1.823288      1.115818
       BuyAndHold   -4.716687      0.000000
ESN    RSI           0.415986      0.111540
       MACD          9.597073      3.704820
       SMA           1.823288      1.016170
       EMA           1.823288      1.016170
       BuyAndHold   -4.576711      0.000000

In [134]:
# Reset the index to use Plotly Express
Result_reset = Result.reset_index()

# Create the bar plot using Plotly Express
fig = px.bar(Result_reset, x="Values", y="Return [%]", color="Strategy", barmode="group",
             title="Return [%] by Strategy")

# Show the plot
fig.show()

In [136]:
# Create the bar plot using Plotly Express
fig = px.bar(Result_reset, x="Values", y="Sharpe Ratio", color="Strategy", barmode="group",
             title="Sharpe Ratio by Strategy")

# Show the plot
fig.show()